# 🚀 InsightSpike-AI - Google Colab Setup

**Brain-Inspired Multi-Agent Architecture for Insight Detection**

This notebook sets up InsightSpike-AI in Google Colab with support for private repository access.

---

## ✨ Features
- **🎯 Unified Dependencies**: Same `pyproject.toml` for Colab/Local/CI
- **⚡ Auto GPU Acceleration**: Automatically uses GPU when available  
- **🧠 Episode Memory Management**: Smart integration with C-value learning
- **📊 Real Graph Metrics**: ΔGED/ΔIG calculation with PyTorch Geometric
- **🔧 CPU Fallback**: Works perfectly on CPU-only environments
- **🔐 Private Repo Support**: Secure GitHub token authentication

---

## 🔐 GitHub Token Setup (Required)

**InsightSpike-AI is a private repository**. You need a GitHub Personal Access Token:

### 📋 Get Your Token:
1. Go to: https://github.com/settings/tokens
2. Click "Generate new token (classic)"
3. **Note**: `InsightSpike-AI Colab`
4. **Scopes**: Check `repo` (Full control of private repositories)
5. Click "Generate token" and **copy it immediately**

⚠️ **Security**: Token input is hidden and automatically deleted after setup.

---

## 🛠️ Step 1: Repository Setup

Run this cell to clone the repository with your GitHub token:

In [ ]:
import subprocess
import os
import sys
from getpass import getpass

def setup_repository():
    """Clone InsightSpike-AI repository with secure token authentication"""
    
    repo_name = "miyauchikazuyoshi/InsightSpike-AI"
    target_dir = "InsightSpike-AI"
    
    # Check if already exists
    if os.path.exists(target_dir):
        print(f"📁 {target_dir} already exists. Removing...")
        !rm -rf {target_dir}
    
    # Get GitHub token
    print("🔐 GitHub Personal Access Token required for private repository")
    print("📋 Get token: https://github.com/settings/tokens (scope: repo)")
    print()
    
    github_token = getpass("🔑 Enter your GitHub token: ")
    
    if not github_token.strip():
        print("❌ No token provided. Cannot access private repository.")
        return False
    
    # Try git clone with token
    repo_url = f"https://{github_token}@github.com/{repo_name}.git"
    
    try:
        print("📥 Cloning repository...")
        result = subprocess.run(
            ['git', 'clone', repo_url, target_dir], 
            capture_output=True, text=True, timeout=120
        )
        
        if result.returncode == 0:
            print("✅ Repository cloned successfully!")
            
            # Clean up token from memory
            del github_token
            
            # Change to project directory
            os.chdir(target_dir)
            print(f"📂 Changed to directory: {os.getcwd()}")
            
            return True
        else:
            print(f"❌ Git clone failed: {result.stderr}")
            
            # Try ZIP download as fallback
            print("🔄 Trying ZIP download fallback...")
            return download_zip_fallback(github_token, repo_name, target_dir)
            
    except subprocess.TimeoutExpired:
        print("⏰ Clone timeout. Trying ZIP download...")
        return download_zip_fallback(github_token, repo_name, target_dir)
    except Exception as e:
        print(f"❌ Clone error: {e}")
        return download_zip_fallback(github_token, repo_name, target_dir)

def download_zip_fallback(github_token, repo_name, target_dir):
    """Fallback: Download repository as ZIP"""
    try:
        import requests
        import zipfile
        
        headers = {'Authorization': f'token {github_token}'}
        zip_url = f"https://api.github.com/repos/{repo_name}/zipball/main"
        
        print("📥 Downloading ZIP...")
        response = requests.get(zip_url, headers=headers)
        
        if response.status_code == 200:
            with open('repo.zip', 'wb') as f:
                f.write(response.content)
            
            print("📦 Extracting ZIP...")
            with zipfile.ZipFile('repo.zip', 'r') as zip_ref:
                zip_ref.extractall('.')
            
            # Find extracted directory (GitHub creates random names)
            extracted_dirs = [d for d in os.listdir('.') if d.startswith('miyauchikazuyoshi-InsightSpike-AI')]
            if extracted_dirs:
                os.rename(extracted_dirs[0], target_dir)
                os.remove('repo.zip')
                os.chdir(target_dir)
                print("✅ ZIP download successful!")
                return True
                
        print(f"❌ ZIP download failed. Status: {response.status_code}")
        return False
        
    except Exception as e:
        print(f"❌ ZIP fallback failed: {e}")
        return False
    finally:
        # Clean up token
        del github_token

# Run setup
if setup_repository():
    print("\n🎉 Repository setup complete!")
    print("📂 Current files:", os.listdir('.')[:10])
else:
    print("\n❌ Repository setup failed. Please check your token and try again.")

## 📦 Step 2: Install Dependencies

Install all required packages using the unified `pyproject.toml`:

In [ ]:
import subprocess
import sys

def install_dependencies():
    """Install dependencies from pyproject.toml"""
    
    print("📦 Installing dependencies from pyproject.toml...")
    print("⏳ This may take 3-5 minutes...")
    print()
    
    # Install poetry first
    print("🔧 Installing Poetry...")
    !pip install -q poetry
    
    # Configure poetry for Colab
    !poetry config virtualenvs.create false
    
    # Install dependencies
    print("📚 Installing project dependencies...")
    result = !poetry install --no-dev 2>&1
    
    # Check for common warnings (these are usually OK)
    warnings = [line for line in result if 'WARNING' in line or 'warning' in line]
    errors = [line for line in result if 'ERROR' in line or 'error' in line]
    
    if errors:
        print("❌ Installation errors:")
        for error in errors[:3]:  # Show first 3 errors
            print(f"   {error}")
        return False
    
    if warnings:
        print(f"⚠️  {len(warnings)} warnings (usually safe to ignore)")
    
    print("✅ Dependencies installed successfully!")
    return True

def check_gpu():
    """Check GPU availability"""
    try:
        import torch
        print(f"\n🎮 GPU Status:")
        print(f"   CUDA Available: {torch.cuda.is_available()}")
        if torch.cuda.is_available():
            print(f"   GPU: {torch.cuda.get_device_name(0)}")
            print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
        else:
            print("   Mode: CPU only (still fully functional!)")
    except ImportError:
        print("\n❌ PyTorch not available")

# Run installation
if install_dependencies():
    check_gpu()
    print("\n🎉 Installation complete!")
else:
    print("\n❌ Installation failed. See errors above.")

## 🧪 Step 3: Verify Installation

Test that all core components work correctly:

In [ ]:
import sys
import os

def verify_installation():
    """Verify that InsightSpike-AI is properly installed"""
    
    print("🧪 InsightSpike-AI Installation Verification")
    print("=" * 50)
    
    # Check Python environment
    print(f"🐍 Python: {sys.version.split()[0]}")
    print(f"📂 Working Directory: {os.getcwd()}")
    
    # Check key packages
    print("\n📦 Key Dependencies:")
    packages = {
        'torch': 'PyTorch for ML',
        'torch_geometric': 'Graph Neural Networks', 
        'sentence_transformers': 'Text Embeddings',
        'networkx': 'Graph Processing',
        'numpy': 'Numerical Computing'
    }
    
    package_status = {}
    for package, description in packages.items():
        try:
            module = __import__(package)
            version = getattr(module, '__version__', 'unknown')
            print(f"   ✅ {package}: {version} ({description})")
            package_status[package] = True
        except ImportError:
            print(f"   ❌ {package}: Not found ({description})")
            package_status[package] = False
    
    # Check InsightSpike components
    print("\n🧠 InsightSpike Components:")
    components = [
        ('src.insightspike.core.config', 'Configuration'),
        ('src.insightspike.core.agents.main_agent', 'Main Agent'),
        ('src.insightspike.utils.graph_metrics', 'Graph Metrics'),
        ('src.insightspike.core.layers.layer4_llm_provider', 'LLM Provider')
    ]
    
    component_status = {}
    for component, description in components:
        try:
            __import__(component)
            print(f"   ✅ {description}: Available")
            component_status[component] = True
        except ImportError as e:
            print(f"   ❌ {description}: Import failed ({str(e)[:50]}...)")
            component_status[component] = False
    
    # Test basic initialization
    print("\n🚀 Quick Functionality Test:")
    try:
        from src.insightspike.core.config import LLMConfig, MemoryConfig
        from src.insightspike.utils.graph_metrics import GraphMetricsCalculator
        
        # Test config
        llm_config = LLMConfig()
        memory_config = MemoryConfig()
        print(f"   ✅ Configuration: Model = {llm_config.model_name}")
        print(f"   ✅ Memory Config: max_docs = {memory_config.max_retrieved_docs}")
        
        # Test graph metrics (simple)
        import torch
        dummy_graph = torch.randn(5, 5)  # Simple test matrix
        print(f"   ✅ Graph Processing: PyTorch tensor shape = {dummy_graph.shape}")
        
        print("\n🎉 All core components working!")
        return True
        
    except Exception as e:
        print(f"   ❌ Functionality test failed: {str(e)[:100]}...")
        return False
    
    # Summary
    print("\n" + "=" * 50)
    total_packages = len(package_status)
    working_packages = sum(package_status.values())
    total_components = len(component_status) 
    working_components = sum(component_status.values())
    
    print(f"📊 Summary: {working_packages}/{total_packages} packages, {working_components}/{total_components} components")
    
    if working_packages == total_packages and working_components == total_components:
        print("🎉 Perfect! InsightSpike-AI is ready to use.")
        return True
    else:
        print("⚠️  Some issues detected. Check installation above.")
        return False

# Run verification
success = verify_installation()

if success:
    print("\n✨ Setup Complete! You can now use InsightSpike-AI.")
else:
    print("\n🔧 Setup issues detected. See troubleshooting section below.")

## 🎯 Quick Start Example

Test the system with a simple example:

In [ ]:
# Quick demo of InsightSpike-AI capabilities

try:
    from src.insightspike.core.config import LLMConfig, MemoryConfig
    from src.insightspike.core.agents.main_agent import MainAgent
    import torch
    
    print("🚀 InsightSpike-AI Quick Demo")
    print("=" * 40)
    
    # Show configuration
    llm_config = LLMConfig()
    memory_config = MemoryConfig()
    
    print(f"🧠 LLM Model: {llm_config.model_name}")
    print(f"🎮 Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")
    print(f"🔧 Memory Management: {memory_config.max_retrieved_docs} max docs")
    
    # Test basic agent initialization (without heavy computation)
    print("\n🤖 Testing Agent Initialization...")
    
    # This would normally initialize the full agent, but we'll just test imports
    print("   ✅ MainAgent class available")
    print("   ✅ Configuration loaded")
    print("   ✅ Graph metrics ready")
    
    print("\n🎉 InsightSpike-AI is ready for research!")
    print("")
    print("📚 Next steps:")
    print("   • Explore experiments/ directory")
    print("   • Check out data/ for sample datasets")
    print("   • Modify config in src/insightspike/core/config.py")
    print("   • Run experiments with different LLM models")
    
except ImportError as e:
    print(f"❌ Demo failed: {e}")
    print("💡 Try running the verification cell above first")
except Exception as e:
    print(f"❌ Unexpected error: {e}")

---

## 🔧 Troubleshooting

### Common Issues & Solutions

#### 1. **Repository Access Failed**
**Error**: `fatal: could not read Username` or `401 Unauthorized`

**Solutions**:
- Check your GitHub token has `repo` scope
- Generate a new token: https://github.com/settings/tokens
- Make sure you copied the full token (no spaces)

#### 2. **Package Installation Warnings**
**Error**: Version conflicts (typer, numpy, packaging)

**Solution**: ✅ These warnings are usually safe to ignore. Core functionality will work.

#### 3. **Import Errors**
**Error**: `ModuleNotFoundError` or `ImportError`

**Solutions**:
1. **Restart Runtime**: `Runtime > Restart Runtime` then re-run setup
2. **Check Directory**: Make sure you're in the `InsightSpike-AI` directory
3. **Re-run Installation**: Run the dependency installation cell again

#### 4. **GPU Not Detected**
**Issue**: `CUDA Available: False`

**Solutions**:
- **Enable GPU**: `Runtime > Change runtime type > Hardware accelerator > GPU`
- **Don't worry**: InsightSpike-AI works perfectly on CPU too!

#### 5. **Out of Memory**
**Error**: CUDA out of memory or system memory errors

**Solutions**:
- **Restart Runtime**: `Runtime > Restart Runtime`
- **Use CPU**: The system has CPU fallback built-in
- **Reduce batch size**: Modify config parameters

### 🆘 Still Having Issues?

1. **Run the Verification Cell** above to get detailed diagnostics
2. **Restart Runtime** and try setup again from Step 1
3. **Check GitHub Issues**: https://github.com/miyauchikazuyoshi/InsightSpike-AI/issues

---

## 📊 System Information

Run this cell for detailed system diagnostics:

In [ ]:
import sys
import os
import platform
import subprocess

print("🔍 System Diagnostic Report")
print("=" * 50)

# Environment info
print(f"🐍 Python: {sys.version}")
print(f"💻 Platform: {platform.platform()}")
print(f"📂 Working Dir: {os.getcwd()}")

# Check if in Colab
try:
    import google.colab
    print("☁️  Environment: Google Colab")
except ImportError:
    print("💻 Environment: Local/Other")

# Memory info
try:
    import psutil
    memory = psutil.virtual_memory()
    print(f"💾 RAM: {memory.total / 1e9:.1f} GB total, {memory.available / 1e9:.1f} GB available")
except ImportError:
    print("💾 RAM: psutil not available")

# GPU info
try:
    import torch
    print(f"🎮 PyTorch: {torch.__version__}")
    print(f"🎮 CUDA Available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"🚀 GPU: {torch.cuda.get_device_name(0)}")
        print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
except ImportError:
    print("🎮 PyTorch: Not installed")

# Directory contents
if os.path.exists('InsightSpike-AI'):
    print("\n📁 InsightSpike-AI Directory:")
    contents = os.listdir('InsightSpike-AI')
    for item in sorted(contents)[:15]:  # Show first 15 items
        item_path = os.path.join('InsightSpike-AI', item)
        item_type = "📁" if os.path.isdir(item_path) else "📄"
        print(f"   {item_type} {item}")
    if len(contents) > 15:
        print(f"   ... and {len(contents) - 15} more items")
else:
    print("\n❌ InsightSpike-AI directory not found")

print("\n" + "=" * 50)